In [14]:
import csv
import cv2
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.impute import KNNImputer
import os

INPUT_PATH = "/kaggle/input/tabular-playground-series-aug-2022"

In [15]:
# reading .csv files with pandas 
train_df = pd.read_csv(f'{INPUT_PATH}/train.csv')
train_df = train_df.reset_index().set_index('id').drop('index', axis=1)

x_test = pd.read_csv(f'{INPUT_PATH}/test.csv')
x_test = x_test.reset_index().set_index('id').drop('index', axis=1)

# spliting x & y, training & validation
x_train=train_df.drop(['failure'],axis=1)
y_train=train_df.failure

# missing value flag (trying)
x_train['m_3_missing'] = x_train.measurement_3.isna().astype(int)
x_train['m_5_missing'] = x_train.measurement_5.isna().astype(int)

x_test['m_3_missing'] = x_test.measurement_3.isna().astype(int)
x_test['m_5_missing'] = x_test.measurement_5.isna().astype(int)

# We fill the missing values
for f in x_train.columns:
    if f == 'loading' or f.startswith('measurement'):
        features.append(f)
imputer = KNNImputer(n_neighbors=50)
imputer.fit(x_train[features])
x_train[features] = imputer.transform(x_train[features])

for f in x_test.columns:
    if f == 'loading' or f.startswith('measurement'):
        features.append(f)
imputer = KNNImputer(n_neighbors=50)
imputer.fit(x_test[features])
x_test[features] = imputer.transform(x_test[features])

In [16]:
# deriving mean (trying)
x_train['area'] = x_train['attribute_2'] * x_train['attribute_3']
x_train['avg'] = x_train[['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
                          'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12',
                          'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16']].mean(axis=1)

x_test['area'] = x_test['attribute_2'] * x_test['attribute_3']
x_test['avg'] = x_test[['measurement_3', 'measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
                          'measurement_8', 'measurement_9', 'measurement_10', 'measurement_11', 'measurement_12',
                          'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16']].mean(axis=1)

# removing some columns with high distribution mismatch(trying)
x_train = x_train.drop(['measurement_3','measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
                      'measurement_8', 'measurement_9','measurement_10', 'measurement_11', 'measurement_12',
                      'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'], axis=1)

x_test = x_test.drop(['measurement_3','measurement_4', 'measurement_5', 'measurement_6', 'measurement_7',
                      'measurement_8', 'measurement_9','measurement_10', 'measurement_11', 'measurement_12',
                      'measurement_13', 'measurement_14', 'measurement_15', 'measurement_16'], axis=1)

x_train = x_train.drop(['product_code'], axis=1)
x_train = x_train.drop(['attribute_3', 'attribute_2', 'attribute_1', 'attribute_0'], axis=1)
x_test = x_test.drop(['product_code'], axis=1)
x_test = x_test.drop(['attribute_3', 'attribute_2', 'attribute_1', 'attribute_0'], axis=1)

# standardize
x_train[['area', 'avg', 'loading', 'measurement_17', 'measurement_0', 'measurement_1','measurement_2',
          'm_3_missing', 'm_5_missing']] = StandardScaler().fit_transform(x_train[['loading', 'area', 'avg',
          'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'm_3_missing', 'm_5_missing']])

x_test[['area', 'avg', 'loading', 'measurement_17', 'measurement_0', 'measurement_1','measurement_2',
          'm_3_missing', 'm_5_missing']] = StandardScaler().fit_transform(x_test[['loading', 'area', 'avg',
          'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'm_3_missing', 'm_5_missing']])

In [17]:
model = LogisticRegression(penalty='l1', C=0.01, solver='liblinear', random_state=1)
model.fit(x_train, y_train)
pred_test = model.predict_proba(x_test)[:,1]

In [18]:
submission = pd.DataFrame({'id': x_test.index,
                           'failure': pred_test})
submission.to_csv('submission.csv', index=False)